In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
# 항목을 고유 숫자로 라벨링함
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor

In [2]:
# area_a = pd.read_csv('../csv/capital/tn_visit_area_info_방문지정보_A.csv', low_memory=False)
# area_b = pd.read_csv('../csv/east/tn_visit_area_info_방문지정보_B.csv', low_memory=False)
# area_c = pd.read_csv('../csv/west/tn_visit_area_info_방문지정보_C.csv', low_memory=False)
# area_d = pd.read_csv('../csv/jeju/tn_visit_area_info_방문지정보_D.csv', low_memory=False)
#
# travel_a = pd.read_csv('../csv/capital/tn_travel_여행_A.csv', low_memory=False)
# travel_b = pd.read_csv('../csv/east/tn_travel_여행_B.csv', low_memory=False)
# travel_c = pd.read_csv('../csv/west/tn_travel_여행_C.csv', low_memory=False)
# travel_d = pd.read_csv('../csv/jeju/tn_travel_여행_D.csv', low_memory=False)
#
# traveller_a = pd.read_csv('../csv/capital/tn_traveller_master_여행객_Master_A.csv')
# traveller_b = pd.read_csv('../csv/east/tn_traveller_master_여행객_Master_B.csv')
# traveller_c = pd.read_csv('../csv/west/tn_traveller_master_여행객_Master_C.csv')
# traveller_d = pd.read_csv('../csv/jeju/tn_traveller_master_여행객_Master_D.csv')

In [3]:
# area_all = pd.concat([area_a, area_b, area_c, area_d], axis=0, ignore_index=True)
# travel_all = pd.concat([travel_a, travel_b, travel_c, travel_d], axis=0, ignore_index=True)
# traveller_all = pd.concat([traveller_a, traveller_b, traveller_c, traveller_d], axis=0, ignore_index=True)

In [4]:
# area_all.to_csv('../csv/tn_visit_area_all.csv', index=False)
# travel_all.to_csv('../csv/tn_visit_travel_all.csv', index=False)
# traveller_all.to_csv('../csv/tn_visit_traveller_all.csv', index=False)

In [5]:
# traveller_all.info()

area_all = pd.read_csv('../../csv/tn_visit_area_all.csv', low_memory=False)
travel_all = pd.read_csv('../../csv/tn_visit_travel_all.csv', low_memory=False)
travelers_all = pd.read_csv('../../csv/tn_visit_traveller_all.csv', low_memory=False)

In [6]:
df = pd.merge(area_all, travel_all, how='left', on='TRAVEL_ID')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164195 entries, 0 to 164194
Data columns (total 32 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   VISIT_AREA_ID         164195 non-null  int64  
 1   TRAVEL_ID             164195 non-null  object 
 2   VISIT_ORDER           164195 non-null  int64  
 3   VISIT_AREA_NM         164195 non-null  object 
 4   VISIT_START_YMD       164195 non-null  object 
 5   VISIT_END_YMD         164195 non-null  object 
 6   ROAD_NM_ADDR          117306 non-null  object 
 7   LOTNO_ADDR            129903 non-null  object 
 8   X_COORD               134598 non-null  object 
 9   Y_COORD               134598 non-null  object 
 10  ROAD_NM_CD            170 non-null     float64
 11  LOTNO_CD              10871 non-null   object 
 12  POI_ID                69397 non-null   object 
 13  POI_NM                69397 non-null   object 
 14  RESIDENCE_TIME_MIN    151341 non-null  float64
 15  

In [7]:
df = pd.merge(df, travelers_all, how='left', on='TRAVELER_ID')

In [8]:
df_filter = df[~df['TRAVEL_MISSION_CHECK'].isnull()].copy()  # 'TRAVEL_MISSION_CHECK' 컬럼에 결측치가 없는 행만을 복사

# 미션 = 여행목적으로 보여짐 한 컬럼에 최대 3개가 있지만 일단 맨 처음 1개만 사용
df_filter.loc[:, 'TRAVEL_MISSION_INT'] = df_filter['TRAVEL_MISSION_CHECK'].str.split(';').str[0].astype(int)

In [9]:
df_filter = df_filter[[
    'GENDER',  # 성별
    'AGE_GRP',  # 연령대
    # 여행스타일
    'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3', 'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6',
    'TRAVEL_STYL_7', 'TRAVEL_STYL_8',
    'TRAVEL_MOTIVE_1',  # 여행동기 (3까지 있음)
    'TRAVEL_COMPANIONS_NUM',  # 동반자 수
    'TRAVEL_MISSION_INT',  # 여행 목적 최우선순위
    'VISIT_AREA_NM',  # 방문지명
    'DGSTFN',  # 만족도
]]

df_filter = df_filter.dropna()

In [10]:
df_filter.info()

<class 'pandas.core.frame.DataFrame'>
Index: 86908 entries, 0 to 164194
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   GENDER                 86908 non-null  object 
 1   AGE_GRP                86908 non-null  float64
 2   TRAVEL_STYL_1          86908 non-null  float64
 3   TRAVEL_STYL_2          86908 non-null  float64
 4   TRAVEL_STYL_3          86908 non-null  float64
 5   TRAVEL_STYL_4          86908 non-null  float64
 6   TRAVEL_STYL_5          86908 non-null  float64
 7   TRAVEL_STYL_6          86908 non-null  float64
 8   TRAVEL_STYL_7          86908 non-null  float64
 9   TRAVEL_STYL_8          86908 non-null  float64
 10  TRAVEL_MOTIVE_1        86908 non-null  float64
 11  TRAVEL_COMPANIONS_NUM  86908 non-null  float64
 12  TRAVEL_MISSION_INT     86908 non-null  int32  
 13  VISIT_AREA_NM          86908 non-null  object 
 14  DGSTFN                 86908 non-null  float64
dtypes: flo

In [11]:
df_filter

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,VISIT_AREA_NM,DGSTFN
0,남,30.0,4.0,4.0,2.0,4.0,6.0,5.0,6.0,6.0,1.0,0.0,5,프로방스마을,4.0
2,여,40.0,4.0,3.0,6.0,3.0,2.0,2.0,7.0,2.0,6.0,0.0,28,병점역 1호선,4.0
3,여,30.0,4.0,4.0,4.0,3.0,4.0,4.0,5.0,7.0,1.0,1.0,6,더현대서울,5.0
4,남,30.0,7.0,1.0,1.0,1.0,7.0,7.0,2.0,4.0,1.0,0.0,2,강릉중앙시장,4.0
5,남,20.0,6.0,1.0,1.0,3.0,4.0,6.0,3.0,2.0,3.0,1.0,11,청계천,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164187,여,20.0,1.0,2.0,2.0,4.0,5.0,2.0,5.0,6.0,1.0,2.0,28,함덕촐래밥상,5.0
164189,남,30.0,3.0,3.0,3.0,4.0,3.0,3.0,4.0,6.0,2.0,1.0,26,문개항아리 조천본점,5.0
164192,남,40.0,2.0,2.0,5.0,1.0,1.0,3.0,2.0,3.0,1.0,1.0,2,메종글래드제주,3.0
164193,여,30.0,1.0,1.0,1.0,3.0,1.0,1.0,3.0,7.0,1.0,1.0,24,옹진농협하나로마트 덕적점,5.0


In [12]:
# 변환할 열 목록
columns_to_convert = ['AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
                      'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
                      'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM']

# 성별도 int형으로 변환
df_filter['GENDER'] = df_filter['GENDER'].replace({'남': 1, '여': 0}).astype('int32')

# integer로 변환 (int32 사용으로 메모리 절약)
df_filter[columns_to_convert] = df_filter[columns_to_convert].astype('int32')

df_filter.dtypes

C:\Users\user\AppData\Local\Temp\ipykernel_12672\4244724412.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filter['GENDER'] = df_filter['GENDER'].replace({'남': 1, '여': 0}).astype('int32')


GENDER                     int32
AGE_GRP                    int32
TRAVEL_STYL_1              int32
TRAVEL_STYL_2              int32
TRAVEL_STYL_3              int32
TRAVEL_STYL_4              int32
TRAVEL_STYL_5              int32
TRAVEL_STYL_6              int32
TRAVEL_STYL_7              int32
TRAVEL_STYL_8              int32
TRAVEL_MOTIVE_1            int32
TRAVEL_COMPANIONS_NUM      int32
TRAVEL_MISSION_INT         int32
VISIT_AREA_NM             object
DGSTFN                   float64
dtype: object

In [20]:
df_filter

,GENDER,AGE_GRP,TRAVEL_STYL_1,TRAVEL_STYL_2,TRAVEL_STYL_3,TRAVEL_STYL_4,TRAVEL_STYL_5,TRAVEL_STYL_6,TRAVEL_STYL_7,TRAVEL_STYL_8,TRAVEL_MOTIVE_1,TRAVEL_COMPANIONS_NUM,TRAVEL_MISSION_INT,VISIT_AREA_NM,DGSTFN
0,1,30,4,4,2,4,6,5,6,6,1,0,5,프로방스마을,4.0
2,0,40,4,3,6,3,2,2,7,2,6,0,28,병점역 1호선,4.0
3,0,30,4,4,4,3,4,4,5,7,1,1,6,더현대서울,5.0
4,1,30,7,1,1,1,7,7,2,4,1,0,2,강릉중앙시장,4.0
5,1,20,6,1,1,3,4,6,3,2,3,1,11,청계천,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164187,0,20,1,2,2,4,5,2,5,6,1,2,28,함덕촐래밥상,5.0
164189,1,30,3,3,3,4,3,3,4,6,2,1,26,문개항아리 조천본점,5.0
164192,1,40,2,2,5,1,1,3,2,3,1,1,2,메종글래드제주,3.0
164193,0,30,1,1,1,3,1,1,3,7,1,1,24,옹진농협하나로마트 덕적점,5.0


In [21]:
le = LabelEncoder()

# 숫자로 방문지명 라벨링
encoded_area = le.fit_transform(df_filter['VISIT_AREA_NM'])

In [22]:
X = df_filter.drop(columns=['DGSTFN'])
X['VISIT_AREA_NM'] = encoded_area

Y = df_filter['DGSTFN']
visit_areas = df_filter['VISIT_AREA_NM']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [23]:
rd = RandomForestClassifier()
y_pred = rd.fit(X_train, y_train)

In [24]:
rd_score = rd.score(X_test, y_test)
rd_score

0.5382196141602424

In [25]:
y_pred = rd.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mse

1.0441069305411728

In [26]:
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=100)

In [27]:
knn_score = knn.score(X_test, y_test)
knn_score

0.4553752924481264

In [28]:
nn = NearestNeighbors(n_neighbors=20, metric='euclidean')
nn.fit(X_train, y_train)

NearestNeighbors(metric='euclidean', n_neighbors=20)

In [29]:
scaler = StandardScaler()
X_scaled_train = scaler.fit_transform(X_train)
X_scaled_test = scaler.transform(X_test)

rfr = RandomForestRegressor()
rfr.fit(X_scaled_train, y_train)

RandomForestRegressor()

In [30]:
rfr.score(X_scaled_test, y_test)

0.04976290638280023

In [31]:
X_scaled = scaler.fit_transform(X)

nn = NearestNeighbors(n_neighbors=20, metric='euclidean')
nn.fit(X_scaled)

NearestNeighbors(metric='euclidean', n_neighbors=20)

In [32]:
# 추천 함수
def recommend_travel_areas(gender, age, styl_1, styl_2, styl_3, styl_4, styl_5, styl_6, styl_7, styl_8,
                          motive, companions, mission):
    user_input = [[gender, age, styl_1, styl_2, styl_3, styl_4, styl_5, styl_6, styl_7, styl_8,
                   motive, companions, mission]]
    user_scaled = scaler.transform(user_input)
    distances, indices = nn.kneighbors(user_scaled)
    recommendations = df_filter.iloc[indices[0]][['VISIT_AREA_NM', 'DGSTFN']]
    top_10 = recommendations.drop_duplicates(subset=['VISIT_AREA_NM'])\
                            .sort_values(by='DGSTFN', ascending=False)\
                            .head(10)
    return top_10

In [33]:
# X: 사용자 특징만 (13개 피처)
X = df_filter[['GENDER', 'AGE_GRP', 'TRAVEL_STYL_1', 'TRAVEL_STYL_2', 'TRAVEL_STYL_3',
               'TRAVEL_STYL_4', 'TRAVEL_STYL_5', 'TRAVEL_STYL_6', 'TRAVEL_STYL_7',
               'TRAVEL_STYL_8', 'TRAVEL_MOTIVE_1', 'TRAVEL_COMPANIONS_NUM', 'TRAVEL_MISSION_INT']]

# 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# NearestNeighbors 모델
nn = NearestNeighbors(n_neighbors=20, metric='euclidean')
nn.fit(X_scaled)

# 추천 함수
def recommend_travel_areas(gender, age, styl_1, styl_2, styl_3, styl_4, styl_5, styl_6, styl_7, styl_8,
                          motive, companions, mission):
    user_input = pd.DataFrame([[gender, age, styl_1, styl_2, styl_3, styl_4, styl_5, styl_6, styl_7, styl_8,
                                motive, companions, mission]],
                              columns=X.columns)  # 피처 이름 포함
    user_scaled = scaler.transform(user_input)
    distances, indices = nn.kneighbors(user_scaled)
    recommendations = df_filter.iloc[indices[0]][['VISIT_AREA_NM', 'DGSTFN']]
    top_10 = recommendations.drop_duplicates(subset=['VISIT_AREA_NM'])\
                            .sort_values(by='DGSTFN', ascending=False)\
                            .head(10)
    return top_10

# Precision@10 평가 함수
def evaluate_precision_at_10(gender, age, styl_1, styl_2, styl_3, styl_4, styl_5, styl_6, styl_7, styl_8,
                            motive, companions, mission):
    recommendations = recommend_travel_areas(gender, age, styl_1, styl_2, styl_3, styl_4, styl_5,
                                            styl_6, styl_7, styl_8, motive, companions, mission)
    relevant = len(recommendations[recommendations['DGSTFN'] >= 4])  # 만족도 4 이상
    precision = relevant / 10  # 10개 중 관련 있는 비율
    return precision

# 실행
precision = evaluate_precision_at_10(1, 30, 4, 4, 2, 4, 6, 5, 6, 6, 1, 0, 5)
print("Precision@10:", precision)

Precision@10: 1.0


In [34]:
# 샘플 입력 (30대 남성, 야외 스포츠 목적 등)
result = recommend_travel_areas(1, 30, 4, 4, 2, 4, 6, 5, 6, 6, 1, 0, 5)
print("추천 여행지 10개:")
print(result)

추천 여행지 10개:
       VISIT_AREA_NM  DGSTFN
28239           지혜의숲     5.0
13659     아이노스 출판단지점     5.0
18718  롯데프리미엄아울렛 파주점     5.0
50613     경주 오소육전&메밀     5.0
57509          뷰델카라반     5.0
48124        경주문무대왕릉     4.0
20461          전쟁기념관     4.0
58840    스타벅스 경주대릉원점     4.0
30579             우직     4.0
45720           경주오릉     4.0


In [35]:
df_filter[df_filter['DGSTFN'] == 1].index

Index([    83,    851,    999,   1183,   1242,   1297,   1500,   1865,   2461,
         3206,
       ...
       161662, 161832, 161916, 162094, 162245, 162421, 162995, 163041, 163172,
       163256],
      dtype='int64', length=985)

In [36]:
df_filter[df_filter['DGSTFN'] == 2].index

Index([    18,    271,    773,    820,    825,    909,    917,   1134,   1176,
         1197,
       ...
       163617, 163718, 163793, 163886, 163970, 164008, 164020, 164153, 164157,
       164179],
      dtype='int64', length=1862)

In [37]:
df_filter[df_filter['DGSTFN'] == 3].index

Index([    12,     25,     28,     70,     75,     89,     94,    102,    117,
          123,
       ...
       164083, 164086, 164117, 164118, 164123, 164125, 164145, 164147, 164176,
       164192],
      dtype='int64', length=11622)

In [38]:
df_filter[df_filter['DGSTFN'] == 4].index

Index([     0,      2,      4,      5,      8,     10,     21,     22,     23,
           29,
       ...
       164162, 164163, 164165, 164168, 164170, 164173, 164177, 164181, 164182,
       164194],
      dtype='int64', length=32560)

In [39]:
df_filter[df_filter['DGSTFN'] == 5].index

Index([     3,      6,     11,     13,     19,     24,     44,     46,     50,
           53,
       ...
       164142, 164158, 164166, 164167, 164172, 164178, 164186, 164187, 164189,
       164193],
      dtype='int64', length=39879)